In [1]:
import os
import pandas as pd
import formatting as fm
import numpy as np

In [2]:
"""
This notebook formats the data files from the different devices to be consistent
It produces a csv file for each device to be used in the merging of user data
Columns space_id and faculty_name are added so that it can correspond with that of the users'
"""

"\nThis notebook formats the data files from the different devices to be consistent\nIt produces a csv file for each device to be used in the merging of user data\nColumns space_id and faculty_name are added so that it can correspond with that of the users'\n"

# spaces database

In [3]:
#spaces database
sde1 = pd.read_excel("Spaces-database.xlsx").drop(columns=["coordinates.1", "notes", "reference_images"]).rename(columns={"SenSING_sensors_id" : "senSING_sensor_id"})
sde2 = pd.read_excel("Spaces-database.xlsx", sheet_name=1).drop([29, 30]).drop(columns=["reference_images", "notes"])
sde4 = pd.read_excel("Spaces-database.xlsx", sheet_name=2).drop([57, 58, 59]).drop(columns=["notes", "reference_images"])
spaces_database = pd.concat([sde1, sde2, sde4])
spaces_database = spaces_database.astype({"space_id" : "Int64"})
#will be referenced from the other sensor data (awair, bms, ubibot, purpleair)

In [4]:
spaces_database.to_csv("spaces_database_z.csv")
spaces_database

,space_id,space_name,floor,room_name,coordinates,awair-device_uuid,BMS-endpoint,ubibot,webid,senSING_sensor_id,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,building,Unnamed: 11,Unnamed: 12,deploy_sensor?
0,101,STAFF_LOUNGE,2.0,SDE1,"[[103.7703660134668,1.2972224509501018],[103.7...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102,EXHIBITION_HALL_LEVEL1,1.0,SDE1,"[[103.7702607402286,1.297297163005382],[103.77...",awair-omni_16477,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103,EXHIBITION_HALL_LEVEL2,2.0,SDE1,"[[103.77030702114178,1.2972821886595796],[103....",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,104,GALLERY,2.0,SDE1,"[[103.77046004395163,1.297124457910897],[103.7...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105,CORRIDOR_2-1,2.0,SDE1,"[[103.77032387667981,1.297040433016008],[103.7...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,<NA>,E&P,5.0,SDE4-05-,NaN,awair-omni_8764,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
54,<NA>,STUDIO EAST,5.0,SDE4-05-,NaN,awair-omni_6501,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
55,<NA>,STUDIO WEST,5.0,SDE4-05-,NaN,awair-omni_11757,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
56,57,CORRIDOR_AT_LEVEL6,6.0,NaN,"[[103.77011885702791,1.2970468013392633],[103....",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN


# purpleair dataset

In [5]:
#purpleair dataset
def purpleair_dataset():
    container = []
    for file in os.listdir("purpleair"):
        data = pd.read_csv(f"{os.getcwd()}/purpleair/{file}")
        data["faculty_name"] = f"SDE{file[13]}"
        container.append(data)
    df = pd.concat(container)
    df = fm.format_time(df)
    df = df.set_index("time")
    return df

purpleair_dataset = purpleair_dataset()

# ubibot dataset

In [6]:
#Ubibot dataset
ubibot_names = ["time", "Temperature(℃)", "Humidity(%)", "Light(lux)", "Voltage(V)", "WIFI RSSI(dBm)", "GSM RSSI(dBm)", "EXT1 Temperature(℃)", "EXT2 Temperature(℃)", "RS485 Temperature(℃)", "RS485 Humidity(%)"]
#some ubibot have different column names for some reason (but the numbers represent the same measurement)

def merge_all_ubibot_files():
    """
    merges all ubibot files by id
    """
    dic = {}
    all_data = []
    main_dir = f"{os.getcwd()}/ubibot"
    main_files = os.listdir("ubibot")
    for folder in main_files:
        path = f"{main_dir}/{folder}"
        for file in os.listdir(path):
            if "csv" in file:
                name = file[:12]
                if name not in dic:
                    dic[name] = []
                df = pd.read_csv(f"{path}/{file}")
                df.columns = ubibot_names
                dic[name].append(df)
    for name, data in dic.items():
        df = pd.concat(list(data))
        df["space_id"] = float(name[9:12])
        #df.to_csv(f"{name}_merged.csv")
        all_data.append(df)
    df_all = fm.format_time(pd.concat(all_data))
    df_all = df_all.astype({"space_id" : "Int64"})
    return df_all.set_index("time")

ubibot_dataset = merge_all_ubibot_files()

# bms dataset

In [7]:
#BMS dataset
bms_dataset = pd.concat(map(lambda x: pd.read_csv(f"{os.getcwd()}/bms/{x}", low_memory=False), os.listdir("bms")))
bms_dataset.columns.values[0] = "time"
bms_dataset = fm.format_time(bms_dataset.drop(bms_dataset.filter(regex='Awair').columns, axis=1))
bms_dataset = bms_dataset.set_index("time")

In [8]:
def get_bms_endpoint():
    lst = []
    for col in list(bms_dataset.columns):
        for i in range(len(col)):
            start = 28
            end = 28
            if col[i] == "|":
                end = i
            lst.append(col[start:end])
    return list(filter(lambda x: x != "", lst))

bms_endpoint = get_bms_endpoint()

In [9]:
bms_endpoint

['VAV2-3',
 'VAV2-4',
 'VAV2-5',
 'VAV2-6',
 'VAV2-18',
 'VAV2-20',
 'VAV2-7N8',
 'VAV2-9N27',
 'VAV2-22',
 'VAV2-17',
 'VAV2-16',
 'VAV2-12']

In [10]:
def get_bms_space_id():
    """
    includes the corresponding space_id with the bms endpoint
    """
    lst = []
    for b in bms_endpoint:
        space_id = int(spaces_database.loc[spaces_database["BMS-endpoint"] == b]["space_id"])
        lst.append(f"{b}|Temp at space_id: {space_id}")
    return lst

In [11]:
new_bms_cols = get_bms_space_id()
new_bms_cols
#changed the column names to include the corresponding space_id

['VAV2-3|Temp at space_id: 201',
 'VAV2-4|Temp at space_id: 201',
 'VAV2-5|Temp at space_id: 201',
 'VAV2-6|Temp at space_id: 201',
 'VAV2-18|Temp at space_id: 202',
 'VAV2-20|Temp at space_id: 203',
 'VAV2-7N8|Temp at space_id: 204',
 'VAV2-9N27|Temp at space_id: 204',
 'VAV2-22|Temp at space_id: 206',
 'VAV2-17|Temp at space_id: 207',
 'VAV2-16|Temp at space_id: 207',
 'VAV2-12|Temp at space_id: 208']

In [12]:
bms_dataset.columns = new_bms_cols

# awair dataset

In [13]:
awair_id_names = pd.read_csv("awair_inventory.csv")["device_uuid"].values

In [14]:
awair_id_names

array(['awair-omni_6867', 'awair-omni_6785', 'awair-omni_11757',
       'awair-omni_8764', 'awair-omni_6882', 'awair-omni_6598',
       'awair-omni_12659', 'awair-omni_6519', 'awair-r2_2661',
       'awair-omni_8862', 'awair-omni_11745', 'awair-omni_16202',
       'awair-omni_11694', 'awair-omni_6501', 'awair-omni_2051',
       'awair-omni_12665', 'awair-omni_12672', 'awair-omni_6545',
       'awair-omni_6672', 'awair-omni_2047', 'awair-omni_6514',
       'awair-omni_12622', 'awair-omni_6452', 'awair-omni_6890',
       'awair-omni_12575', 'awair-omni_6674', 'awair-omni_6509',
       'awair-omni_12596', 'awair-omni_258', 'awair-omni_12986',
       'awair-omni_16379', 'awair-omni_16477', 'awair-omni_16572',
       'awair-omni_16314', 'awair-omni_16193', 'awair-omni_16197',
       'awair-omni_16351', 'awair-omni_16517'], dtype=object)

In [15]:
spaces = spaces_database.rename(columns={"awair-device_uuid" : "device_uuid"}).reset_index()

In [16]:
def spaces_with_awair():
    """
    from the spaces database, filters out spaces with no awair device
    """
    container = []
    for i in awair_id_names:
        container.append(spaces.loc[spaces["device_uuid"] == i])
    return pd.concat(container).reset_index().drop(columns=["index", "level_0"])

In [17]:
spaces_awair = spaces_with_awair()

In [18]:
spaces_awair

,space_id,space_name,floor,room_name,coordinates,device_uuid,BMS-endpoint,ubibot,webid,senSING_sensor_id,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,building,Unnamed: 11,Unnamed: 12,deploy_sensor?
0,52,DESIGN STUDIO WEST,5.0,SDE4-05-06,NaN,awair-omni_11757,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
1,<NA>,STUDIO WEST,5.0,SDE4-05-,NaN,awair-omni_11757,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
2,<NA>,E&P,5.0,SDE4-05-,NaN,awair-omni_8764,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
3,47,GREEN BUILDINGS TECHNOLOGIES Lab,2.0,SDE4-02-01,NaN,awair-omni_12659,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
4,16,DESIGN STUDIO1 (ID),5.0,SDE4-05-06,"[[103.77007338481161,1.2969628410478435],[103....",awair-omni_8862,"VAV-L5-06, 07, 08, 09, 10, 11, 12, 13 &14 TS",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
5,46,DESIGN STUDIO 2 (JC OPENBLUE),5.0,SDE4-05-03 (West),NaN,awair-omni_6501,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
6,<NA>,STUDIO EAST,5.0,SDE4-05-,NaN,awair-omni_6501,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
7,37,ARCHITECTURE STUDIO 2,4.0,SDE4-04-03,"[[103.77011976645349,1.2970461172680814],[103....",awair-omni_2051,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
8,25,URBAN GREENERY LAB,2.0,SDE4-02-04,"[[103.77029829633938,1.296892305647276],[103.7...",awair-omni_12665,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
9,26,SOFT MODELLING ASSEMBLY WORKSHOP,2.0,SDE4-02-02,"[[103.7704100057768,1.2969282082444522],[103.7...",awair-omni_12672,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN


In [19]:
def merge_awair():
    container = []
    folder = os.listdir("awair")
    awair_inventory = pd.read_csv("awair_inventory.csv")
    for file in folder:
        name = fm.reformat(file)[:-4]
        df = pd.read_csv(f"{os.getcwd()}/awair/{file}")
        for i in range(len(awair_inventory["display_name"])):
            if fm.reformat(awair_inventory["display_name"][i]) == name:
                df["device_uuid"] = awair_inventory["device_uuid"][i]
                container.append(df)
    dataset = pd.concat(container)
    dataset.columns.values[0] = "time"
    dataset = fm.format_time(dataset)
    dataset["space_id"] = pd.NA
    dataset = dataset.reset_index()
    for i in range(len(dataset)):
        for j in range(len(spaces_awair)):
            if dataset["device_uuid"][i] == spaces_awair["device_uuid"][j]:
                dataset.at[i, "space_id"] = spaces_awair.at[j, "space_id"]
    return dataset.drop(columns="index")

awair_dataset = merge_awair().set_index("time")

# Sensing dataset

In [20]:
sensing_id_names = list(map(lambda x: x[5:-4], os.listdir("sensing")))

In [21]:
sensing_id_names

['ASB005221418',
 'ASB102181830',
 'ASB004651418',
 'ASB007881418',
 'ASB005431418',
 'ASB005151418',
 'ASB105891831',
 'ASB004101418',
 'ASB104571831',
 'ASB006031418',
 'ASB004611418',
 'ASB116981902',
 'ASB105281831',
 'ASB104791831',
 'ASB102871830',
 'ASB102461830']

In [22]:
def spaces_with_sensing():
    """
    from the spaces database, filters out spaces with no sensing device
    """
    container = []
    for i in sensing_id_names:
        container.append(spaces.loc[spaces["senSING_sensor_id"] == i])
    return pd.concat(container).reset_index().drop(columns=["index", "level_0"])

In [23]:
spaces_sensing = spaces_with_sensing()
spaces_sensing

,space_id,space_name,floor,room_name,coordinates,device_uuid,BMS-endpoint,ubibot,webid,senSING_sensor_id,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,building,Unnamed: 11,Unnamed: 12,deploy_sensor?
0,124,SEMINAR_ROOM6,4.0,SDE1,"[[103.77060344296319,1.297024277321114],[103.7...",NaN,NaN,NaN,NaN,ASB005221418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,213,EXECUTIVE ROOM4,3.0,NaN,"[[103.77116815707154,1.2973660189932872],[103....",NaN,N.A,NaN,N.A,ASB102181830,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE2,NaN,NaN,NaN
2,129,SEMINAR_ROOM2,4.0,SDE1,"[[103.77056902354377,1.2972144240290646],[103....",NaN,NaN,1.0,NaN,ASB004651418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,204,WINDTUNNEL UNIT/BUILDING SYSTEMS & DIAGNOSTICS...,2.0,NaN,"[[103.77119379955133,1.2971933132695455],[103....",NaN,VAV2-7N8,NaN,F1EmPuLYsdTuuUqI0OYHjD6iEwjEYm6nnB6BGEThhm2vYo...,ASB007881418,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE2,NaN,NaN,NaN
4,41,TERRACE,4.0,NaN,"[[103.77036386007256,1.2969284822921026],[103....",NaN,NaN,NaN,NaN,ASB005431418,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
5,125,SEMINAR_ROOM5,4.0,SDE1,"[[103.7705828937012,1.2971082072880762],[103.7...",NaN,NaN,3.0,NaN,ASB005151418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,<NA>,BEEHUB2,3.0,SDE4-03-02,"[[103.77047490389816,1.296921078613579],[103.7...",awair-omni_12575,VAV-L3-02&03 TS,NaN,NaN,ASB105891831,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE4,NaN,NaN,NaN
7,214,SEMINAR ROOM,3.0,NaN,"[[103.7711033038795,1.297174261954538],[103.77...",NaN,N.A,7.0,N.A,ASB004101418,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE2,NaN,NaN,NaN
8,206,CORRIDOR2-1,2.0,NaN,"[[103.77124410673702,1.2975714341202575],[103....",NaN,VAV2-22,10.0,F1EmPuLYsdTuuUqI0OYHjD6iEwaEYm6nnB6BGEThhm2vYo...,ASB104571831,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE2,NaN,NaN,NaN
9,202,BUILDING STRUCTURES BUILDING MATERIALS LAB,2.0,NaN,"[[103.77107284917429,1.2973233213766946],[103....",NaN,VAV2-18,8.0,F1EmPuLYsdTuuUqI0OYHjD6iEwiUYm6nnB6BGEThhm2vYo...,ASB006031418,...,NaN,NaN,NaN,NaN,NaN,NaN,SDE2,NaN,NaN,NaN


In [24]:
def merge_sensing():
    container = []
    folder = os.listdir("sensing")
    for file in folder:
        sensor_id = file[5:-4]
        df = pd.read_csv(f"{os.getcwd()}/sensing/{file}")
        for row in range(len(spaces_sensing)):
            if spaces_sensing.at[row, "senSING_sensor_id"] == sensor_id:
                df["space_id"] = spaces_sensing.at[row, "space_id"]
        container.append(df)
    return pd.concat(container).set_index("time")

sensing_dataset = merge_sensing()

# datasets
these data should have their space_id included, I use **Int64 (with big I)** to represent space_id because int is not compatible with NaN and float will not work with merge_asof "by" condition

In [25]:
bms_dataset.to_csv("bms_dataset_z.csv")
bms_dataset

,VAV2-3|Temp at space_id: 201,VAV2-4|Temp at space_id: 201,VAV2-5|Temp at space_id: 201,VAV2-6|Temp at space_id: 201,VAV2-18|Temp at space_id: 202,VAV2-20|Temp at space_id: 203,VAV2-7N8|Temp at space_id: 204,VAV2-9N27|Temp at space_id: 204,VAV2-22|Temp at space_id: 206,VAV2-17|Temp at space_id: 207,VAV2-16|Temp at space_id: 207,VAV2-12|Temp at space_id: 208
time,,,,,,,,,,,,
2021-02-01 00:00:00+08:00,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail
2021-02-01 00:05:00+08:00,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail
2021-02-01 00:10:00+08:00,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail
2021-02-01 00:15:00+08:00,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail
2021-02-01 00:20:00+08:00,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail,Comm Fail
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-27 23:40:00+08:00,26.42000008,26.15999985,26.15999985,26.32999992,25.90999985,26.25,25.55999947,27.10000038,24.54000092,24.70999908,25.64999962,27.19000053
2021-04-27 23:45:00+08:00,26.42000008,26.15999985,26.15999985,26.32999992,25.90999985,26.25,25.55999947,27.10000038,24.54000092,24.70999908,25.64999962,27.19000053
2021-04-27 23:50:00+08:00,26.42000008,26.15999985,26.15999985,26.32999992,25.90999985,26.25,25.55999947,27.10000038,24.54000092,24.70999908,25.64999962,27.19000053


In [26]:
ubibot_dataset.to_csv("ubibot_dataset_z.csv")
ubibot_dataset

,Temperature(℃),Humidity(%),Light(lux),Voltage(V),WIFI RSSI(dBm),GSM RSSI(dBm),EXT1 Temperature(℃),EXT2 Temperature(℃),RS485 Temperature(℃),RS485 Humidity(%),space_id
time,,,,,,,,,,,
2021-03-08 16:04:00+08:00,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,123
2021-03-08 16:04:00+08:00,NaN,NaN,NaN,5.794865,NaN,NaN,NaN,NaN,NaN,NaN,123
2021-03-08 16:04:00+08:00,NaN,NaN,NaN,5.805032,NaN,NaN,NaN,NaN,NaN,NaN,123
2021-03-08 16:04:00+08:00,NaN,NaN,NaN,5.761152,NaN,NaN,NaN,NaN,NaN,NaN,123
2021-03-08 16:03:00+08:00,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,123
...,...,...,...,...,...,...,...,...,...,...,...
2021-03-05 20:12:00+08:00,NaN,NaN,550.880005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219
2021-03-05 20:12:00+08:00,28.711754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219
2021-03-05 20:10:00+08:00,NaN,NaN,NaN,6.134134,NaN,NaN,NaN,NaN,NaN,NaN,219


In [27]:
purpleair_dataset.to_csv("purpleair_dataset_z.csv")
purpleair_dataset

,purpleair.0.3_um_count,purpleair.0.5_um_count,purpleair.1.0_um_count,purpleair.10.0_um_count,purpleair.2.5_um_count,purpleair.5.0_um_count,purpleair.humidity,purpleair.pm1.0,purpleair.pm10.0,purpleair.pm2.5,purpleair.pressure,purpleair.temperature,faculty_name
time,,,,,,,,,,,,,
2021-03-01 16:30:00+08:00,4781.67,1394.98,230.78,1.18,10.82,2.22,44.0,23.58,40.78,35.05,1005.87,96.00,SDE2
2021-03-01 16:30:00+08:00,4158.91,1263.98,242.64,0.52,11.14,1.14,NaN,22.02,37.93,33.88,NaN,NaN,SDE2
2021-03-01 16:40:00+08:00,4754.05,1392.65,241.96,0.63,14.54,2.53,44.0,23.12,41.67,35.42,1005.78,96.00,SDE2
2021-03-01 16:40:00+08:00,4245.74,1279.44,238.32,0.00,9.88,1.44,NaN,21.95,37.68,33.47,NaN,NaN,SDE2
2021-03-01 16:50:00+08:00,5040.95,1481.05,277.25,1.02,21.25,2.46,48.0,24.16,46.09,38.72,1005.63,96.00,SDE2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-20 16:00:00+08:00,1715.78,508.69,72.42,0.22,2.89,0.22,NaN,8.84,13.05,12.82,NaN,NaN,SDE4
2021-05-20 16:10:00+08:00,1393.47,405.07,57.22,0.00,2.80,0.27,51.0,7.24,10.29,10.16,1004.88,31.11,SDE4
2021-05-20 16:10:00+08:00,1340.95,400.42,55.82,0.00,1.78,0.24,NaN,6.87,9.87,9.73,NaN,NaN,SDE4


In [28]:
awair_dataset.to_csv("awair_dataset_z.csv")
awair_dataset

,score,temp(°C),humid,co2,voc,pm25,light,noise,device_uuid,space_id
time,,,,,,,,,,
2021-03-01 00:00:00+08:00,74.6,26.8,78.0,407.1,179.3,11.3,0.0,64.5,awair-omni_12672,26
2021-03-01 00:15:00+08:00,75.0,26.8,77.9,405.0,181.2,10.1,0.0,64.4,awair-omni_12672,26
2021-03-01 00:30:00+08:00,75.0,26.8,77.9,405.1,174.2,9.6,0.0,64.4,awair-omni_12672,26
2021-03-01 00:45:00+08:00,75.0,26.7,77.6,408.8,178.6,9.4,0.0,64.4,awair-omni_12672,26
2021-03-01 01:00:00+08:00,75.0,26.7,77.8,404.9,177.7,9.5,0.0,64.4,awair-omni_12672,26
...,...,...,...,...,...,...,...,...,...,...
2021-05-08 03:15:00+08:00,75.2,26.1,73.1,732.2,20.0,4.7,0.0,52.3,awair-omni_16351,141
2021-05-08 03:30:00+08:00,75.1,26.1,72.9,732.7,20.0,4.8,0.0,52.4,awair-omni_16351,141
2021-05-08 03:45:00+08:00,75.4,26.1,72.8,725.3,20.0,4.4,0.0,52.5,awair-omni_16351,141


In [29]:
sensing_dataset.to_csv("sensing_dataset_z.csv")
sensing_dataset

,senSING.humidity,senSING.light,senSING.noise,senSING.temperature,space_id
time,,,,,
2021-03-26T09:43:22.000+08:00,65.2354,33,43,23.1719,124
2021-03-29T17:21:27.000+08:00,70.6123,32,46,21.5291,124
2021-03-29T17:22:27.000+08:00,69.6852,21,43,21.8128,124
2021-03-29T17:23:27.000+08:00,68.6468,21,43,22.0778,124
2021-03-29T17:24:27.000+08:00,68.1722,21,43,22.2113,124
...,...,...,...,...,...
2021-05-14T16:31:26.000+08:00,91.5060,21,60,28.2638,19
2021-05-14T16:32:26.000+08:00,91.2309,22,57,28.4422,19
2021-05-14T16:33:26.000+08:00,85.9710,20,61,28.3525,19
